In [1]:
import yfinance as yf
from datetime import date, timedelta

from Demos.win32console_demo import window_size

# Test dates
amazon_stock = yf.download('AMZN', start='2020-01-01', end='2024-01-01')

[*********************100%***********************]  1 of 1 completed


In [3]:
import pandas as pd

window_size = 5

X = []
y = []

# Definir X e y
for i in range(len(amazon_stock) - window_size):
    X.append(amazon_stock['Close'].values[i:i + window_size])  # Ventana de entrada
    y.append(amazon_stock['Close'].values[i + window_size])    # Siguiente valor de 'Close' como objetivo


# Convertir a DataFrame para entrenamiento
X_train = pd.DataFrame(X)
y_train = pd.Series(y)

print("Tamaño del conjunto de entrenamiento:", X_train.shape, y_train.shape)

ValueError: Must pass 2-d input. shape=(1001, 5, 1)

In [16]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib
from sklearn.metrics import  root_mean_squared_error
import numpy as np

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
from sklearn.metrics import mean_squared_error
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "Amazon Stock validation",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    
    import joblib
    joblib.dump(booster, 'models/xgboost_model_amz.pkl')
    print("Modelo guardado como 'db_models/xgboost_model_amz.pkl'")
        
    mlflow.log_artifact('models/xgboost_model_amz.pkl', artifact_path="model")

2024/11/02 16:44:54 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 2.1.1, but the installed version is 2.1.2. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.


[0]	validation-rmse:1.15434                           
[1]	validation-rmse:0.62232                           
[2]	validation-rmse:0.63582                           
[3]	validation-rmse:0.62250                           
[4]	validation-rmse:0.64304                           
[5]	validation-rmse:0.64064                           
[6]	validation-rmse:0.63587                           
[7]	validation-rmse:0.63597                           
[8]	validation-rmse:0.64825                           
[9]	validation-rmse:0.65517                           
[10]	validation-rmse:0.65191                          
  0%|          | 0/10 [00:30<?, ?trial/s, best loss=?]

2024/11/02 16:45:27 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/02 16:45:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:45:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/02 16:45:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: 

[0]	validation-rmse:1.36180                                                     
[1]	validation-rmse:0.87515                                                     
[2]	validation-rmse:0.83315                                                     
[3]	validation-rmse:0.76810                                                     
[4]	validation-rmse:0.72748                                                     
[5]	validation-rmse:0.72736                                                     
[6]	validation-rmse:0.72999                                                     
[7]	validation-rmse:0.72024                                                     
[8]	validation-rmse:0.70256                                                     
[9]	validation-rmse:0.67020                                                     
[10]	validation-rmse:0.69205                                                    
[11]	validation-rmse:0.67728                                                    
[12]	validation-rmse:0.68546

2024/11/02 16:45:42 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/02 16:45:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:45:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/02 16:45:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: 

...

[0]	validation-rmse:26.45872                                                    
[1]	validation-rmse:24.99625                                                    
[2]	validation-rmse:23.61995                                                    
[3]	validation-rmse:22.31922                                                    
[4]	validation-rmse:21.08586                                                    
[5]	validation-rmse:19.92097                                                    
[6]	validation-rmse:18.82344                                                    
[7]	validation-rmse:17.78600                                                    
[8]	validation-rmse:16.80185                                                    
[9]	validation-rmse:15.86928                                                    
[10]	validation-rmse:14.99277                                                   
[11]	validation-rmse:14.16599                                                   
[12]	validation-rmse:13.3871

2024/11/02 16:47:09 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/02 16:47:09 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:47:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/02 16:47:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: 

100%|██████████| 10/10 [02:22<00:00, 14.21s/trial, best loss: 0.3074780395785337]
[0]	validation-rmse:26.12061
[1]	validation-rmse:24.36343
[2]	validation-rmse:22.72545
[3]	validation-rmse:21.19469
[4]	validation-rmse:19.77068
[5]	validation-rmse:18.43730
[6]	validation-rmse:17.19487
[7]	validation-rmse:16.03182
[8]	validation-rmse:14.95521
[9]	validation-rmse:13.94442
[10]	validation-rmse:13.00661
[11]	validation-rmse:12.13390
[12]	validation-rmse:11.31917
[13]	validation-rmse:10.56077
[14]	validation-rmse:9.85069
[15]	validation-rmse:9.19134
[16]	validation-rmse:8.57239
[17]	validation-rmse:7.99886
[18]	validation-rmse:7.46436
[19]	validation-rmse:6.96515
[20]	validation-rmse:6.49742
[21]	validation-rmse:6.06275
[22]	validation-rmse:5.65818
[23]	validation-rmse:5.28120
[24]	validation-rmse:4.93035
[25]	validation-rmse:4.60157
[26]	validation-rmse:4.29502
[27]	validation-rmse:4.01056
[28]	validation-rmse:3.74337
[29]	validation-rmse:3.49476
[30]	validation-rmse:3.26495
[31]	validation

2024/11/02 16:47:19 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/11/02 16:47:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:47:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/11/02 16:47:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'models/xgboost_model_amz.pkl'


In [27]:
best_params

{'learning_rate': 0.06769425128151128,
 'max_depth': 20,
 'min_child_weight': 0.8168884846784011,
 'reg_alpha': 0.05802603765092618,
 'reg_lambda': 0.025868224207792168,
 'seed': 42,
 'objective': 'reg:squarederror'}

In [28]:
booster